![cover](./img/cover.png)

In [1]:
#IMPORTING VERY COMMON AND USEFULL PACKAGES
import numpy as np
import pandas as pd
import datetime as dt
import xlrd

#OPTIMIZING
import scipy.optimize as so

#CHARTING
import plotly
import chart_studio.plotly as py
import plotly.offline as po
import plotly.graph_objs as go

#WIDGETING
import ipywidgets as widgets
from ipywidgets import interact
from ipywidgets.embed import embed_minimal_html

#INITIALIZING NOTEBOOK MODE
po.init_notebook_mode(connected=True)

In [2]:
#Definig a function that convert dates from Excel formats
def xl2date(excel_date):
    if isinstance(excel_date, int):
        return dt.datetime(*xlrd.xldate_as_tuple(excel_date,0))
    elif isinstance(excel_date,pd.Series) or isinstance(excel_date, list):
        dates = [dt.datetime(*xlrd.xldate_as_tuple(date,0)) for date in excel_date]
        return dates      

# Defining a function that compute the bond price for a given YtM
def bond_price(ytm, face_value, time_to_expiry, annual_rate, pay_frequency=2):
    freq = float(pay_frequency)
    periods = time_to_expiry*freq
    coupon = face_value * ( annual_rate/freq)
    dt = [(1+i)/freq for i in range(int(periods))]
    return sum([coupon/(1+ytm)**t for t in dt]) + face_Value/(1+ytm)**(dt[-1])

#Defining a function that compute the bond Yield to Maturity for a given market price
def bond_ytm(price, face_value, time_to_expiry, annual_rate, pay_frequency=2, guess = 0.05):
    freq = float(pay_frequency)
    periods = time_to_expiry*freq
    coupon = face_value * ( annual_rate/freq)
    dt = [(1+i)/freq for i in range(int(periods))]
    
    ytm_func = lambda y : sum([coupon/(1+y)**(t) for t in dt]) + face_value/(1+y)**(dt[-1]) - price
    
    return so.newton(ytm_func, guess)

### <center>Yield-to-Maturity</center>
The **Yield-to-Maturity** (**YtM**) is the theoretical **internal rate of return** earned by an investor who buys the bond today at the market price.
It is defined as the **constant discount rate** at which the sum of all future cash flows from the bond is equal to the current price of the bond.

$$ P^{MKT} (t) = \sum_{i=1}^N \frac{CashFlow (T_i)}{\left(1+y \right)^{T_i -t}} = \sum_{i=1}^N \frac{FaceValue \cdot CouponRate \cdot (T_i -T_{i-1})}{\left(1+y \right)^{T_i -t}} + \frac{FaceValue}{\left(1+y \right)^{T_N -t}}$$

In [3]:
btp = pd.read_csv('btp_italia.csv', sep='|')
btp['Maturity'] = xl2date(btp['Maturity'])
btp_italia = pd.DataFrame()
btp_italia['ISIN']=btp['ISIN']
btp_italia['Description']=btp['Description']
btp_italia['Maturity']=btp['Maturity']
btp_italia['Rate']=btp['Rate']
btp_italia['PayFrequency']=btp['PayFrequency']
btp_italia['Price']=btp['Price']

In [4]:
btp_italia.head(30)

,ISIN,Description,Maturity,Rate,PayFrequency,Price
0,IT0004536949,"BTP Mar-20, 4.25",2020-03-01,0.0425,2,100.2180
1,IT0005107708,"BTP May-20, 0.7",2020-05-01,0.0070,2,100.2295
2,IT0005250946,"BTP Jun-20, 0.35",2020-06-15,0.0035,2,100.2385
3,IT0004594930,"BTP Sep-20, 4",2020-09-01,0.0400,2,102.3805
4,IT0005285041,"BTP Oct-20, 0.2",2020-10-15,0.0020,2,100.3705
5,IT0005142143,"BTP Nov-20, 0.65",2020-11-01,0.0065,2,100.7190
6,IT0004634132,"BTP Mar-21, 3.75",2021-03-01,0.0375,2,104.2660
7,IT0005330961,"BTP Apr-21, 0.05",2021-04-15,0.0005,2,100.3770
8,IT0004966401,"BTP May-21, 3.75",2021-05-01,0.0375,2,104.8930
9,IT0005175598,"BTP Jun-21, 0.45",2021-06-01,0.0045,2,100.9240


In [10]:
#Create a scatter plot from BTP prices
min_maty = min(btp['Maturity'])
min_xRange = dt.datetime(min_maty.year-1, min_maty.month, min_maty.day)
max_maty = max(btp['Maturity'])
max_xRange = dt.datetime(max_maty.year+1, max_maty.month, max_maty.day)

btp_price = go.Scatter(x=btp['Maturity'],y=btp['Price'],
                       hovertemplate='<br><b>Price</b>: %{y:.2f}<br>'+
                       '<b>%{text}</b><extra></extra>',
                       text=btp['Description'],
                       name='Bonds price', mode='markers', marker=dict(color='Red', size=5),
                       line=dict(color='Red', width=1, dash='dot'))

prices = go.FigureWidget([btp_price])

#Figure Title
prices['layout']['title'] = {'text':'Prices','font':{'size': 25},'x':0.5,'xanchor':'center'}
#Figure x axis
prices['layout']['xaxis'] = {'range':[min_xRange,max_xRange],
                             'title': 'Maturity', 'titlefont':{'size': 10}}
#Figure y axis
prices['layout']['yaxis'] = {'range':[90,160],'fixedrange': False, 'title': 'Price',
                             'titlefont':{'size': 10},'hoverformat': '.2f'}

prices['layout']['showlegend'] = False
prices['layout']['height'] = 800
prices['layout']['width'] = 1600
#prices['layout']['autosize']=True

prices['layout']['updatemenus']= list([
    dict(type="buttons",
         active=-1,
         buttons=list([
            dict(label = 'Markers',
                 method = 'update',
                 args = [{'mode':'markers'}]),
            dict(label = 'Lines',
                 method = 'update',
                 args = [{'mode':'lines'}]),
            dict(label = 'Markers+Lines',
                 method = 'update',
                 args = [{'mode':'markers+lines'}])]),
        direction='right',
        pad = {'r': 10, 't': 87},
        showactive = True,
        x = 0.5,
        xanchor = 'center',
        y = 0.05,
        yanchor = 'top'
    )
])

#Create a scatter plot from BTP YtM
btp_yield = go.Scatter(x=btp['Maturity'],y=btp['Yield'],
                       hovertemplate='<br><b>Yield</b>: %{y:.4p}<br>'+
                       '<b>%{text}</b><extra></extra>',
                       text=btp['Description'],
                       name='Bonds YtM', mode='markers', marker=dict(color='Green', size=5),
                       line=dict(color='Green', width=1, dash='dot'))


yields = go.FigureWidget([btp_yield])

#Figure Title
yields['layout']['title'] = {'text':'Yield-to-Maturity','font':{'size': 25},'x':0.5,'xanchor':'center'}
#Figure x axis
yields['layout']['xaxis'] = {'range':[min_xRange,max_xRange],
                             'title': 'Maturity', 'titlefont':{'size': 15}}
#Figure y axis
yields['layout']['yaxis'] = {'range':[-0.005,0.0225],
                             'fixedrange': False,
                             'title': 'Yield-to-Maturity',
                             'tickformat': '.2p',
                             'titlefont':{'size': 15},
                             'hoverformat': '.4p'}

yields['layout']['showlegend'] = False
yields['layout']['height'] = 800
yields['layout']['width'] = 1600
#yields['layout']['autosize']=False

yields['layout']['updatemenus']= list([
    dict(type="buttons",
         active=-1,
         buttons=list([
            dict(label = 'Markers',
                 method = 'update',
                 args = [{'mode':'markers'}]),
            dict(label = 'Lines',
                 method = 'update',
                 args = [{'mode':'lines'}]),
            dict(label = 'Markers+Lines',
                 method = 'update',
                 args = [{'mode':'markers+lines'}])]),
        direction='right',
        pad = {'r': 10, 't': 87},
        showactive = True,
        x = 0.5,
        xanchor = 'center',
        y = 0.05,
        yanchor = 'top'
    )
])

## <center>Why yields (instead of prices)?</center>

In [11]:
po.iplot(prices)

In [12]:
po.iplot(yields)